<a href="https://colab.research.google.com/github/daniel-gomm/kd-documents/blob/IMRaD/TrainingIMRaDModels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 2.8 MB 8.4 MB/s 
     |████████████████████████████████| 636 kB 77.7 MB/s 
     |████████████████████████████████| 3.3 MB 73.0 MB/s 
     |████████████████████████████████| 895 kB 66.4 MB/s 
     |████████████████████████████████| 50 kB 8.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from transformers import BertForSequenceClassification
from transformers import BertModel, BertConfig
import random
import numpy as np
import operator
from sklearn.metrics import f1_score

from transformers import AutoTokenizer, AutoModel
from transformers import RobertaTokenizer
from transformers import RobertaConfig, RobertaModel
from transformers import AutoModelForSequenceClassification, AutoTokenizer


In [3]:
import json
import glob
import os
import csv
from nltk.tokenize import sent_tokenize
import nltk
import bs4
from bs4 import BeautifulSoup
from transformers import RobertaTokenizer
from transformers import RobertaConfig, RobertaModel
from transformers import RobertaTokenizer, RobertaForSequenceClassification

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Where to find the training data

In [4]:
AdressTrainingData="/content/drive/MyDrive/Seminar_KD_aifb/csv/FinishedDataCsv/withoutMethodParsing.csv"
pathHelper2="/content/drive/MyDrive/Seminar_KD_aifb/csv/help2.csv"

This method is used to fine-tune a specific model on our training data. You will need to change the adress to save the models to a new adress.

In [14]:
def CreateModel(path,tokenizerName,tokenizerString,modelName,modelPath, cased,roberta,adressToSaveModel): #Params: path:path of training data, tokenizerName for example BertTokenizer, tokenizer string: for example "bert-base-uncased", ModelName:Name of the pretrained model,
                                                                                                          #modelPath:where to find the pretrained model, cased: whether to use a cased tokenizer-->True:lower cased tokenizer, roberta: is roberta used?, adressToSaveModel
  label_dict = {}
  label_dict["intro"]=0
  label_dict["related"]=1
  label_dict["method"]=2
  label_dict["results"]=3
  label_dict["discussion"]=4

  #read training data csv as pandas
  df = pd.read_csv(path)
  df['Section'].value_counts()
  possible_labels = df.Section.unique()
  df['label'] = df.Section.replace(label_dict)

  #split the dataset into evaluation and training
  X_train, X_val, y_train, y_val = train_test_split(df.index.values,df.label.values,test_size=0.03,random_state=42, stratify=df.label.values)
  df['data_type'] = ['not_set']*df.shape[0]
  df.loc[X_train, 'data_type'] = 'train'
  df.loc[X_val, 'data_type'] = 'val'

  #create arrays with the data, yields an error otherwise 
  array1=(df[df.data_type=='train'].Sentence.values).tolist()
  for i in range (0,len(array1)):
    array1[i]=str(array1[i])

  array2=(df[df.data_type=='val'].Sentence.values).tolist()
  for i in range (0,len(array2)):
    array2[i]=str(array2[i])

  #initialize tokenizer
  if (roberta==True):
    tokenizer = tokenizerName.from_pretrained(tokenizerString)
  else: 
    tokenizer = tokenizerName.from_pretrained(tokenizerString,do_lower_case=cased)
  #encode the train and test data with the tokenizer so that it can be used to train the transformer
  encoded_data_train = tokenizer.batch_encode_plus(
      array1, 
      add_special_tokens=True, 
      return_attention_mask=True, 
      pad_to_max_length=True, 
      max_length=256, 
      return_tensors='pt'
  )

  encoded_data_val = tokenizer.batch_encode_plus(
      array2, 
      add_special_tokens=True, 
      return_attention_mask=True, 
      pad_to_max_length=True, 
      max_length=256, 
      return_tensors='pt'
  )

  input_ids_train = encoded_data_train['input_ids']
  attention_masks_train = encoded_data_train['attention_mask']
  labels_train = torch.tensor(df[df.data_type=='train'].label.values)

  input_ids_val = encoded_data_val['input_ids']
  attention_masks_val = encoded_data_val['attention_mask']
  labels_val = torch.tensor(df[df.data_type=='val'].label.values)

  dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
  dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

  #load the specified pretrained model
  model = modelName.from_pretrained(modelPath,
                                                        num_labels=len(label_dict),
                                                        output_attentions=False, 
                                                        output_hidden_states=False)

  batch_size = 10

  dataloader_train = DataLoader(dataset_train, 
                                sampler=RandomSampler(dataset_train), 
                                batch_size=batch_size)

  dataloader_validation = DataLoader(dataset_val, 
                                    sampler=SequentialSampler(dataset_val), 
                                    batch_size=batch_size)
  optimizer = AdamW(model.parameters(),
                    lr=1e-5, 
                    eps=1e-8)
                    
  epochs = 5

  scheduler = get_linear_schedule_with_warmup(optimizer, 
                                              num_warmup_steps=0,
                                              num_training_steps=len(dataloader_train)*epochs)
  #micro f1 as measurement
  def f1_score_func(preds, labels):
      preds_flat = np.argmax(preds, axis=1).flatten()
      labels_flat = labels.flatten()
      return f1_score(labels_flat, preds_flat, average='micro')
  #accuracy as measurement
  def accuracy_per_class(preds, labels):
      label_dict_inverse = {v: k for k, v in label_dict.items()}
      
      preds_flat = np.argmax(preds, axis=1).flatten()
      labels_flat = labels.flatten()

      for label in np.unique(labels_flat):
          y_preds = preds_flat[labels_flat==label]
          y_true = labels_flat[labels_flat==label]
          print(f'Class: {label_dict_inverse[label]}')
          print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

  #load to GPU, a lot faster
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  device = "cuda:0"
  model = model.to(device)
  print(device)
  counterSave=0
  seed_val = 42
  random.seed(seed_val)
  np.random.seed(seed_val)
  torch.manual_seed(seed_val)
  torch.cuda.manual_seed_all(seed_val)

  #actually train the model on the training data in 5 epochs.
  def evaluate(dataloader_val):
      
      model.eval()
      
      loss_val_total = 0
      predictions, true_vals = [], []


      for batch in dataloader_val:
          
          batch = tuple(b.to(device) for b in batch)
          
          inputs = {'input_ids':      batch[0],
                    'attention_mask': batch[1],
                    'labels':         batch[2],
                  }

          with torch.no_grad():        
              outputs = model(**inputs)
              
          loss = outputs[0]
          logits = outputs[1]
          loss_val_total += loss.item()

          logits = logits.detach().cpu().numpy()
          label_ids = inputs['labels'].cpu().numpy()
          predictions.append(logits)
          true_vals.append(label_ids)
      
      loss_val_avg = loss_val_total/len(dataloader_val) 
      
      predictions = np.concatenate(predictions, axis=0)
      true_vals = np.concatenate(true_vals, axis=0)
              
      return loss_val_avg, predictions, true_vals
      
  for epoch in tqdm(range(1, epochs+1)):
      
      model.train()
      counterSave=counterSave+1
      loss_train_total = 0

      progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
      for batch in progress_bar:

          model.zero_grad()
          
          batch = tuple(b.to(device) for b in batch)
          
          inputs = {'input_ids':      batch[0],
                    'attention_mask': batch[1],
                    'labels':         batch[2],
                  }       

          outputs = model(**inputs)
          
          loss = outputs[0]
          loss_train_total += loss.item()
          loss.backward()

          torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

          optimizer.step()
          scheduler.step()
          
          progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
      #Dont save the models. If you want the models to be saved, replace the 5 with a 4 and change the rootToSaveModel string.
      if(counterSave>5):
        torch.save(model.state_dict(), f'{rootToSaveModel}{adressToSaveModel}{epoch}.model')
          
      tqdm.write(f'\nEpoch {epoch}')
      
      loss_train_avg = loss_train_total/len(dataloader_train)            
      tqdm.write(f'Training loss: {loss_train_avg}')
      
      val_loss, predictions, true_vals = evaluate(dataloader_validation)
      val_f1 = f1_score_func(predictions, true_vals)
      tqdm.write(f'Validation loss: {val_loss}')
      tqdm.write(f'F1 Score (Weighted): {val_f1}')

Different paths of models. tokenizers etc.

In [17]:
rootToSaveModel="/content/drive/MyDrive/Seminar_KD_aifb/EvalModelleClean/" #Models will not be saved right now. change this parameter to a folder of your liking where you want to save the models if you want to save them.
BertTokenizerName=BertTokenizer

BertStringUncased="bert-base-uncased"
BertStringCased="bert-base-cased"
BertStringUncasedLarge="bert-large-uncased"

ScibertStringUncased="allenai/scibert_scivocab_uncased"
ScibertStringCased="allenai/scibert_scivocab_cased"


BertModelName=BertForSequenceClassification
ScibertPath="allenai/scibert_scivocab_cased"

adressToSaveModelBertLargeUncased="LargeBertUncased"

adressToSaveModelBertUncased="BertUncased"
adressToSaveModelBertCased="BertCased"

adressToSaveModelSciBertUncased="ScibertUncased"
adressToSaveModelSciBertCased="ScibertCased"

RobertaTokenizer=RobertaTokenizer
RobertaModel=RobertaForSequenceClassification
RobertaString="roberta-base"
adressToSaveModelRoberta="RobertaCased"

5 models are trained, bert case and uncased, scibert cased and uncased aswell as roberta. For the final evaluation, we use bert uncased, scibert uncased and roberta as casing did not seem to improve the perfomance

Train Bert Base Uncased

In [ ]:
CreateModel(AdressTrainingData,BertTokenizerName,BertStringUncased,BertModelName,BertStringUncased,True,False,adressToSaveModelBertUncased)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2204: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSeq

cuda:0


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/679 [00:00<?, ?it/s]

Bert Base Cased

In [ ]:
CreateModel(AdressTrainingData,BertTokenizerName,BertStringCased,BertModelName,BertStringCased,False,False,adressToSaveModelBertCased)

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

KeyboardInterrupt: ignored

Scibert Uncased

In [ ]:
CreateModel(AdressTrainingData,AutoTokenizer,ScibertStringUncased,AutoModelForSequenceClassification,ScibertStringUncased,True,False,adressToSaveModelSciBertUncased)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initiali

cuda:0


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/679 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.782100757548314
Validation loss: 0.5875306342329297
F1 Score (Weighted): 0.8031701003340173


...


Epoch 3
Training loss: 0.31559044009949655
Validation loss: 0.6950706936241615
F1 Score (Weighted): 0.8073699483887976


Epoch 4:   0%|          | 0/679 [00:00<?, ?it/s]


Epoch 4
Training loss: 0.20421775880465845
Validation loss: 0.7632424629603823
F1 Score (Weighted): 0.824169194002205


Epoch 5:   0%|          | 0/679 [00:00<?, ?it/s]


Epoch 5
Training loss: 0.1392378779436806
Validation loss: 0.8404323930430803
F1 Score (Weighted): 0.8144426198683508


Scibert Cased

In [ ]:
CreateModel(AdressTrainingData,BertTokenizerName,ScibertStringCased,BertModelName,ScibertStringCased,False,False,adressToSaveModelSciBertCased)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Some weights of the model checkpoint at allenai/scibert_scivocab_cased were not used when initializi

cuda:0


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/679 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.844944377820692
Validation loss: 0.6223111088786807
F1 Score (Weighted): 0.7873801204618813


...


Epoch 3
Training loss: 0.36893069491185393
Validation loss: 0.7386522785361324
F1 Score (Weighted): 0.7738620378241903


Epoch 4:   0%|          | 0/679 [00:00<?, ?it/s]


Epoch 4
Training loss: 0.24699654918855307
Validation loss: 0.7944018186203071
F1 Score (Weighted): 0.7749080264367106


Epoch 5:   0%|          | 0/679 [00:00<?, ?it/s]


Epoch 5
Training loss: 0.17917230567004702
Validation loss: 0.8065361530475673
F1 Score (Weighted): 0.7840335286857777


Roberta

In [ ]:
CreateModel(AdressTrainingData,RobertaTokenizer,RobertaString,RobertaModel,RobertaString,True,True,adressToSaveModelRoberta)

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


...


Epoch 3
Training loss: 0.4958506122626507
Validation loss: 0.6812043282247725
F1 Score (Weighted): 0.7793595329627148


Epoch 4:   0%|          | 0/679 [00:00<?, ?it/s]


Epoch 4
Training loss: 0.37995280016800703
Validation loss: 0.6584891793983323
F1 Score (Weighted): 0.7958267704758277


Epoch 5:   0%|          | 0/679 [00:00<?, ?it/s]


Epoch 5
Training loss: 0.29386330771757857
Validation loss: 0.7070537694685516
F1 Score (Weighted): 0.809196656724518
